In [1]:
!pip install opencv-python matplotlib imageio tensorflow
#gdown

In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
# import gdown

2024-03-06 14:13:16.835390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 14:13:16.949296: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 14:13:17.670479: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/loca

### Get file names for speaker

In [3]:
speaker = 's1'

folders_to_clean = [speaker]
paths_to_clean = ['/'.join([os.getcwd(),'data/raw_videos', folder]) for folder in folders_to_clean]

In [4]:
paths_to_clean

['/home/jupyter/lip_translate/data/raw_videos/s1']

In [ ]:
# file_names = []
# for folder_path in paths_to_clean:
#     for root, dirs, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith('mpg'):
#                 file_name = file.replace(".mpg", "")
#                 if len(file_names) < 6: #Temporary -- comment out cell below when this is working
#                     file_names.append(file_name)
# file_names

In [6]:
file_names = []
for folder_path in paths_to_clean:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('mpg'):
                file_name = file.replace(".mpg", "")
                file_names.append(file_name)
# file_names

In [ ]:
# align_names = [file.replace('.align', '') for file in os.listdir('/home/jupyter/data/raw_alignments/s11')]
# video_names = [file.replace('.mpg', '') for file in os.listdir('/home/jupyter/data/raw_videos/s11')]

In [ ]:
# align_in_vid = []

# for alignment in align_names:
#     if alignment in video_names:
#         align_in_vid.append(alignment)
#         # print(alignment)

In [ ]:
# vid_in_align = []

# for vid in video_names:
#     if vid in align_names:
#         vid_in_align.append(vid)
#         # print(vid)

In [ ]:
# len(vid_in_align), len(video_names)

In [ ]:
# len(align_in_vid), len(align_names)

In [ ]:
# vid_in_align

### Create function to load video

In [7]:
def load_video(path:str) -> List[float]: #Returns a list of the normalized, greyscaled frames of the video of shape=(75, 46, 140, 1)

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))): 
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        frames.append(frame[190:236,80:220,:]) #There is a better way of doing this
    cap.release()
    
    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std

### Define vocabulary

In [8]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

2024-03-06 14:15:41.489008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-03-06 14:15:41.489036: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-06 14:15:41.489059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20240305-111814): /proc/driver/nvidia/version does not exist
2024-03-06 14:15:41.489324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX

### Define function to get alignments

In [9]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': #Removing the silences
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

### Define function to load data

In [10]:
def load_data(file_name: str): 
    video_path = os.path.join(os.getcwd(),'data/raw_videos', speaker, f'{file_name}.mpg')
    alignment_path = os.path.join(os.getcwd(),'data/raw_alignments', speaker, f'{file_name}.align')
    frames = load_video(video_path).numpy().tolist()
    alignments = load_alignments(alignment_path).numpy().tolist()

    return frames, alignments


In [ ]:
# file_names[0]

In [ ]:
# load_data(file_names[0])

### Build dictionary to store outputs

In [12]:
clean_data = dict.fromkeys(file_names, {'clean_frames':None,
                                        'clean_alignments':None})

# clean_data

In [ ]:
# clean_data['lgis1a']

In [23]:
load_data('lgis1a')

75

In [13]:
def make_dict(file_names):
    for file_name in file_names:
        output_dict = clean_data[file_name]
        output_dict['clean_frames'], output_dict['clean_alignments'] = load_data(file_name)
        clean_data[file_name] = output_dict
    return clean_data
    
        
 

In [14]:
clean_dict = make_dict(file_names)
# clean_dict

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe7b64dba60>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe7b64dba60>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
len(clean_dict.keys())

In [ ]:
# Testing converting eager tensor type to list in order to save to json

# clean_vid = clean_dict['lgis1a'].copy()
# clean_vid['clean_frames'] = clean_vid['clean_frames']
# clean_vid['clean_alignments'] = clean_vid['clean_alignments']

In [ ]:
clean_vid

### Save to a JSON

In [ ]:
# import json

# with open('data/processed_data/test_vid.json', 'w') as fp:
#     json.dump(clean_vid, fp)

In [ ]:
# f = open('data/processed_data/test_vid.json')

# test_vid = json.load(f)

In [ ]:
# len(test_vid['clean_frames'][0][0][0])

In [ ]:
# plt.imshow(test_vid['clean_frames'][0])